In [14]:
import pandas as pd
import numpy as np

import chardet

## Datasets de titulos de las plataformas
- Lectura de los datasets.
- Transformaciones pedidas(5)

##### Verificacion de Encoding

In [15]:
# Funcion Auxiliar para identificar el encoding. 
def detectar_encoding(data):
    '''
    Esta función sirve para detectar los tipo de enconding para distintos formatos
    '''
    with open(data, 'rb') as f:
        enc = chardet.detect(f.read())
    return enc['encoding']  

In [16]:
# Vemos que el dataset no tiene caracteres extraños como para usar otro tipo de encoding. 

print( detectar_encoding('data/amazon_prime_titles.csv'))
print( detectar_encoding('data/disney_plus_titles.csv'))
print( detectar_encoding('data/hulu_titles.csv'))
print( detectar_encoding('data/netflix_titles.csv'))


utf-8
utf-8
utf-8
utf-8


##### Carga de Datos

- Vemos que todos los datasets tienen las mismas columnas. 

In [17]:
df_amazon = pd.read_csv('data/amazon_prime_titles.csv')
df_amazon.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...


In [18]:
df_disney = pd.read_csv('data/disney_plus_titles.csv')
df_disney.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...


In [19]:
df_hulu = pd.read_csv('data/hulu_titles.csv')
df_hulu.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."


In [20]:
df_netflix = pd.read_csv('data/netflix_titles.csv')
df_netflix.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


### Transformaciones

##### Transformación 1: 
Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [21]:
# Insertamos en la primera posicion la columna 'id', que contendra 'Inicial_plataforma' + show_id.

df_amazon.insert(loc = 0, column = 'id', value = 'a' + df_amazon.show_id)
df_disney.insert(loc = 0, column = 'id', value = 'd' + df_disney.show_id)
df_hulu.insert(loc = 0, column = 'id', value = 'h' + df_hulu.show_id)
df_netflix.insert(loc = 0, column = 'id', value = 'n' + df_netflix.show_id)

In [22]:
df_amazon.head(3)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,as2,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,as3,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


##### Transformación 2: 

Los valores nulos del campo rating reemplazaránse por el string “ G” 
(corresponde al rating de madurez: “general para todos los públicos”

In [23]:
# Vemos los valores no nulos de la columna 'rating' del dataset de Amazon
df_amazon.rating.info()

<class 'pandas.core.series.Series'>
RangeIndex: 9668 entries, 0 to 9667
Series name: rating
Non-Null Count  Dtype 
--------------  ----- 
9331 non-null   object
dtypes: object(1)
memory usage: 75.7+ KB


In [24]:
# Completa los valores nulos de la columna 'rating' con el string 'G'. 

df_amazon.rating = np.where(df_amazon.rating.isnull(), 'G', df_amazon.rating)
df_disney.rating = np.where(df_disney.rating.isnull(), 'G', df_disney.rating)
df_hulu.rating = np.where(df_hulu.rating.isnull(), 'G', df_hulu.rating)
df_netflix.rating = np.where(df_netflix.rating.isnull(), 'G', df_netflix.rating)


In [25]:
# Vemos que ya no hay nulos de la columna 'rating' del dataset de Amazon
df_amazon.rating.info()

<class 'pandas.core.series.Series'>
RangeIndex: 9668 entries, 0 to 9667
Series name: rating
Non-Null Count  Dtype 
--------------  ----- 
9668 non-null   object
dtypes: object(1)
memory usage: 75.7+ KB


##### Transformacion 3:

De haber fechas, debe tener el formatoAAAA-mm-dd

In [26]:
# Modificamos la columna 'date_added' con el formato requerido. 
 
df_amazon.date_added = pd.to_datetime(df_amazon['date_added'])
df_disney.date_added = pd.to_datetime(df_disney['date_added'])
df_hulu.date_added = pd.to_datetime(df_hulu['date_added'])
df_netflix.date_added = pd.to_datetime(df_netflix['date_added'])


In [27]:
df_amazon.date_added

0      2021-03-30
1      2021-03-30
2      2021-03-30
3      2021-03-30
4      2021-03-30
          ...    
9663          NaT
9664          NaT
9665          NaT
9666          NaT
9667          NaT
Name: date_added, Length: 9668, dtype: datetime64[ns]

##### Transformacion 4:


Los campos de texto deben estar en minúsculas , sin excepciones

In [28]:

# Función para convertir los campos de texto a minuscula. 
def convertir_minuscula(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.lower()


In [29]:
convertir_minuscula(df_amazon)
convertir_minuscula(df_disney)
convertir_minuscula(df_hulu)
convertir_minuscula(df_netflix)

In [30]:
df_amazon.head(2)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...


##### Transformacion 5:

El campo duracion debe convertirse en dos campos: duration_inty duration_type. El primero será un entero y el segundo una cadena indicando la unidad de medición de duración: min (minutos) o temporada (temporadas)

In [31]:
# Vemos que la columna duration tiene 3 tipo de duracion(min, season, seasons)
df_amazon.duration

0         113 min
1         110 min
2          74 min
3          69 min
4          45 min
          ...    
9663       60 min
9664    4 seasons
9665       90 min
9666     1 season
9667      103 min
Name: duration, Length: 9668, dtype: object

In [32]:
# Estandarizo la columna 'duration', reemplazo 'seasons' por 'season'. 

df_amazon.duration = df_amazon.duration.replace('seasons', 'season', regex = True)
df_disney.duration = df_disney.duration.replace('seasons', 'season', regex = True)
df_hulu.duration = df_hulu.duration.replace('seasons', 'season', regex = True)
df_netflix.duration = df_netflix.duration.replace('seasons', 'season', regex = True)

In [33]:
def separar_columna_duration(df):
    
    df_aux = df.duration.str.split( pat = ' ', expand = True )

    df.insert(loc =11, column = 'duration_int', value = df_aux.iloc[:,0])
    df.insert(loc =12, column = 'duration_type', value = df_aux.iloc[:,1])

    # A los valores nulos de la columna 'duration_int' lo completamos con 0, para poder luego convertir esa columna a entero. 
    df.duration_int.fillna(0, inplace=True)
    
    df['duration_int'] = df['duration_int'].astype('int')




In [34]:
separar_columna_duration(df_amazon)
separar_columna_duration(df_disney)
separar_columna_duration(df_hulu)
separar_columna_duration(df_netflix)


In [35]:
df_amazon.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_int,duration_type,listed_in,description
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,113,min,"comedy, drama",a small fishing village must procure a local d...
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,110,min,"drama, international",a metro family decides to fight a cyber crimin...
2,as3,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,74,min,"action, drama, suspense",after a man discovers his wife is cheating on ...
3,as4,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,69,min,documentary,"pink breaks the mold once again, bringing her ..."
4,as5,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,45,min,"drama, fantasy",teenage matt banting wants to work with a famo...


Eliminamos las columnas que reemplazmos por otras: 'duration' y 'show_id'

In [36]:
# Eliminamos las columnas 'show_id' y 'duration.
df_amazon.drop(['show_id', 'duration'], axis = 'columns', inplace=True)
df_disney.drop(['show_id', 'duration'], axis = 'columns', inplace=True)
df_hulu.drop(['show_id', 'duration'], axis = 'columns', inplace=True)
df_netflix.drop(['show_id', 'duration'], axis = 'columns', inplace=True)


 Concateno los 4 dataframe en uno solo

In [37]:
df_data_titles = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], axis = 0)
df_data_titles.head(2)

,id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110,min,"drama, international",a metro family decides to fight a cyber crimin...


In [38]:
# Exporta a un .csv
df_data_titles.to_csv('data_titles.csv', index = False)

## Dataset de ratings

In [39]:
# Leemos los 8 dataset
df_1 = pd.read_csv('Data/ratings/1.csv')
df_2 = pd.read_csv('Data/ratings/2.csv')
df_3 = pd.read_csv('Data/ratings/3.csv')
df_4 = pd.read_csv('Data/ratings/4.csv')
df_5 = pd.read_csv('Data/ratings/5.csv')
df_6 = pd.read_csv('Data/ratings/6.csv')
df_7 = pd.read_csv('Data/ratings/7.csv')
df_8 = pd.read_csv('Data/ratings/8.csv')




In [40]:
df_data_ratings = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8], axis = 0, ignore_index=True)
df_data_ratings.head(2)

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186


In [41]:
df_data_ratings.rename(columns={"movieId": "id"}, inplace = True)
df_data_ratings.head()

,userId,rating,timestamp,id
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [42]:
# Creo el csv de los 8 ratings
df_data_ratings.to_csv('data_ratings.csv', index = False)

### Calculo del dataframe que contendra el Score. Score sera el promedio de la columna 'rating' agrupados por pelicula 'id'

In [43]:
# Por la cantidad de registros se opto por trabajar esta parte con Spark. Es por eso que se ha creado 
# este modulo para crear una funcion que nos ayudara en el calculo de score. 
import spark 

In [44]:
df_score = spark.create_df_score('data_ratings.csv')
df_score.head()

,id,score
0,ns8641,3.5000
1,as1586,3.4768
2,hs1299,3.5724
3,ns8123,3.5661
4,ns6930,3.5089


## Merge de los 3 dataframes
Se unira los 3 dataframe por la columna 'id' que tienen en comun. 

In [68]:
# Primero unimos 2 dataframes. 
df_data = pd.merge(df_data_titles, df_data_ratings, how = 'left',on = 'id')
df_data.head(2)

,id,type,title,director,cast,country,date_added,release_year,rating_x,duration_int,duration_type,listed_in,description,userId,rating_y,timestamp
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,543,5.0,1059596582
1,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,595,3.0,839937603


In [69]:
# Unimos el ultimo dataframe. El resultado sera un solo dataframe que contendra toda la informacion.
df_total = pd.merge(df_data, df_score, how = 'left', on = 'id')
df_total.head(2)

,id,type,title,director,cast,country,date_added,release_year,rating_x,duration_int,duration_type,listed_in,description,userId,rating_y,timestamp,score
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,543,5.0,1059596582,3.4671
1,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,595,3.0,839937603,3.4671


In [70]:
# Exportamos el dataframe final a un .csv
df_total.to_csv('data_total.csv',  index = False)


In [3]:
# Tambien creamos este .csv que solo contiene los titulos de las plataformas con su score. 
df_data = pd.merge(df_data_titles, df_score, how = 'left',on = 'id')
df_data.head(2)

,id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,score
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,3.4671
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110,min,"drama, international",a metro family decides to fight a cyber crimin...,3.5487


In [8]:
# Exportamos el dataframe final a un .csv
df_data.to_csv('data_score_titles.csv',  index = False)
